In [24]:
import os # For CMG/ECL cmd command prompt execution and Directory generation
import shutil
import random # For reproduction of research
import argparse # For definition of set of arguments
import numpy as np
import itertools
import pickle # For saving simulation results

parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()
# args = parser.parse_args()

args.master_directory = "D:\\Lab_Meeting\\Simulation_Model\\2D_15-by-15\\Simulation\\Bruteforce_search"
# args.simulation_directory = "" # Location of actual simulation of each case happens
# args.simulation_data_file = "" # Location of actual simulation data file
# args.simulation_include_file = "" # Location of actual simulation include file
# args.simulation_log = "" # Location of actual simulation log file
# args.master_simulation_data_file = args.master_directory + "\\2D_15-by-15.dat" # Location of master simulation data file
# args.master_well_placement_template = args.master_directory + "\\2D_15-by-15_WellPlacement.template.inc" # Location of master simulation include file
# args.results_report_command = args.master_directory + "\\2D_15-by-15_Results.rwd" # Location of master CMG Results Report command file
# args.simulation_results_report_command = "" # Location of actual CMG Results Report command file

# # Saving former well location
# args.former_well_location = None

# # Making basic simulation directories
# if not os.path.exists(args.master_directory):
#     os.makedirs(args.master_directory)
well_location = (1,1)
args.former_well_location_string = ""
for i in [(1,1), (1,2)]:
    args.former_well_location_string += f"{i}"

args.simulation_directory = args.master_directory + f"\\{args.former_well_location_string}{well_location}".replace(" ", "")
print(args.simulation_directory)
print(type(args.simulation_directory))

D:\Lab_Meeting\Simulation_Model\2D_15-by-15\Simulation\Bruteforce_search\(1,1)(1,2)(1,1)
<class 'str'>


In [26]:
# For ECL, ".DATA" must not be included for successful simulation run command
# args.simulation_data_file = args.simulation_directory + "\\2D_15-by-15.DATA"
# args.simulation_run_file = args.simulation_data_file[:-5] # For ECL Simulation
# with open("TEST.rwd", 'r') as file_read:
#     lines = file_read.readlines()
#     lines[0] = lines[0].replace("TEST.sr3", "12345")
# with open("TEST.rwd", "w") as file_write:
#     for i in range(len(lines)):
#         file_write.write(lines[i])

string = ["1", "str", "**[#Well_Prod_1]"]
matching = [s for s in string if "**[#Well_Prod_" in s]
print(matching)

['**[#Well_Prod_1]']


In [7]:
index = [i for i in range(1, 226)]

import itertools

test = itertools.permutations(index, 5)

print(list(test))


MemoryError: 